In [162]:
%run Plot\ Utilities.ipynb
import random
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import display, Math, Latex

In [3]:
#
# Given a weight vector, plot the weight vector, the corresponding 
# separator, two random vectors u and v that lie on the separator, 
# and u - v.
#
def Proposition_1(w):
    plt.figure()
    ax = plt.axes()
    ax.set_aspect('equal', 'box')
 
    m, b = weight_to_mb(w)
    
    # Figure out size of plot
    xint = -b/m
    if m < 0.01:
        mx = max(abs(b), 5)
        xlim = (-mx, mx)
    elif m > 5:
        xlim = sorted([(5 * xint - b)/m, (-5 * xint - b)/m])
    else:
        mx = max(abs(b), abs(xint))
        xlim = (-mx, mx)

    mx = 1.2 * max(max(map(abs, w)), max(*xlim), abs(m * xlim[0] + b), abs(m * xlim[1] + b))
    ax.set_xlim(-mx, mx)
    ax.set_ylim(-mx, mx)

    # Choose u
    ux = random.uniform(*xlim)
    uy = m * ux + b
    
    # Choose v that is not too close and not too far
    while True:
        vx = random.uniform(*xlim)
        vy = m * vx + b
        if abs(ux - vx) < mx and abs(uy - vy) < mx and \
            ((abs(ux - vx) > 0.2 * mx) or (abs(uy - vy) > 0.2 * mx)):
            break
    
    x = [ux, vx, ux - vx, w[-2]]
    y = [uy, vy, uy - vy, w[-1]]
        
    plt.quiver([0, 0, 0, 0], [0, 0, 0, 0], x, y,
               angles = 'xy', scale_units='xy', scale=1.,
               color = ['blue', 'blue', 'red', 'red'])
    mbline(m, b)
    
    ax.axhline(y=0, color = 'k', linestyle = '--', linewidth = 0.5)
    ax.axvline(x=0, color = 'k', linestyle = '--', linewidth = 0.5)
    
    plt.text(1.1 * x[2], 1.1 * y[2], 'u-v')
    plt.text(1.1 * x[3], 1.1 * y[3], 'w')

    plt.show()
    
    eqn = r'\begin{eqnarray}'
    for idx in range(4):
        eqn += (r'{\bf %6s} & = & ({\tt %.2f}, {\tt %.2f}) \\ ' % (['u', 'v', 'v - u', 'w'][idx], x[idx], y[idx]))
    eqn += r'\end{eqnarray}'
    display(Math(eqn))

In [2]:
def Proposition_2(w, l = None):
    plt.figure()
    ax = plt.axes()
    ax.set_aspect('equal', 'box')
    
    if l is None:
        l = random.uniform(0, 3)    
    m, b = weight_to_mb(w)

    xlim = (-5, 5)
    ax.set_xlim(*xlim)
    ylim = (m * xlim[0] + b, m * xlim[1] + b)
    ax.set_ylim(*ylim)

    ux = random.uniform(*xlim)
    uy = m * ux + b
    vx = ux + l * w[-2]
    vy = uy + l * w[-1]
    
    mx = max(abs(ux), abs(vx), 5)
    my = max(abs(uy), abs(vy), 5)
    ax.set_xlim(-mx, mx)
    ax.set_ylim(-my, my)
    ax.axhline(y=0, color = 'k', linestyle = '--', linewidth = 0.5)
    ax.axvline(x=0, color = 'k', linestyle = '--', linewidth = 0.5)

    mbline(m, b)
    plt.scatter([ux, vx], [uy, vy], color = 'blue')
        
    v = [vx, vy]
    if len(w) > 2:
        v = [1] + v

    plt.quiver([ux], [uy], [l * w[-2]], [l * w[-1]],
               angles = 'xy', scale_units='xy', scale=1.,
               color = 'red' if np.dot(v, w) <= 0 else 'green')
    plt.show()
    
    eqn = r'\begin{eqnarray}'
    eqn += (r'{\bf u} & = & ({\tt %.2f}, {\tt %.2f}) \\ ' % (ux, uy))
    eqn += (r'\lambda & = & {\tt %.2f} \\' % l)
    eqn += (r'{\bf w} \cdot ({\bf u} + \lambda {\bf w}) & = & {\tt %.2f}' % np.dot(v, w))
    eqn += r'\end{eqnarray}'
    display(Math(eqn))

In [30]:
def positive_misclassified_geometry():
    while True:
        vectors = np.random.rand(2, 2)
        a = 2 * vectors[0] - 1
        b = 2 * vectors[1] - 1
        an = np.linalg.norm(a)
        bn = np.linalg.norm(b)
        c = np.dot(a, b) / (an * bn)
        if c < -0.05 and c > -0.95 and an > 0.25 and bn > 0.25:
            break
    ab = a + b
    
    X = [0, 0, a[0], b[0], 0]
    Y = [0, 0, a[1], b[1], 0]
    U = [a[0], b[0], b[0], a[0], ab[0]]
    V = [a[1], b[1], b[1], a[1], ab[1]]
    plt.figure(figsize=(6, 6))
    ax = plt.axes()
    ax.set_aspect('equal', 'box')
    ax.set_xlim((-1.5, 1.5))
    ax.set_ylim((-1.5, 1.5))
    ax.axhline(y=0, color = 'k', linestyle = '--', linewidth = 0.5)
    ax.axvline(x=0, color = 'k', linestyle = '--', linewidth = 0.5)
    
    plt.quiver(X, Y, U, V, angles = 'xy', scale_units='xy', scale=1.,
              color = ['blue', 'blue', 'black', 'black', 'red'])
    
    plt.text(1.1 * a[0], 1.1 * a[1], 'w')
    plt.text(1.1 * b[0], 1.1 * b[1], 'x')
    plt.text(1.1 * ab[0], 1.1 * ab[1], 'w + x')

In [31]:
def negative_misclassified_geometry():
    while True:
        vectors = np.random.rand(2, 2)
        a = 2 * vectors[0] - 1
        b = 2 * vectors[1] - 1
        an = np.linalg.norm(a)
        bn = np.linalg.norm(b)
        c = np.dot(a, b) / (an * bn)
        if c > 0.05 and c < 0.95 and an > 0.25 and bn > 0.25:
            break
    ab = a - b
    
    X = [0, 0, a[0], b[0], 0]
    Y = [0, 0, a[1], b[1], 0]
    U = [a[0], b[0], b[0], a[0], ab[0]]
    V = [a[1], b[1], b[1], a[1], ab[1]]
    plt.figure(figsize=(6, 6))
    ax = plt.axes()
    ax.set_aspect('equal', 'box')
    ax.set_xlim((-1.5, 1.5))
    ax.set_ylim((-1.5, 1.5))
    ax.axhline(y=0, color = 'k', linestyle = '--', linewidth = 0.5)
    ax.axvline(x=0, color = 'k', linestyle = '--', linewidth = 0.5)
    
    plt.quiver(X, Y, U, V, angles = 'xy', scale_units='xy', scale=1.,
              color = ['blue', 'blue', 'black', 'black', 'red'])
    
    plt.text(1.1 * a[0], 1.1 * a[1], 'w')
    plt.text(1.1 * b[0], 1.1 * b[1], 'x')
    plt.text(1.1 * ab[0], 1.1 * ab[1], 'w - x')

In [32]:
def perceptron_loss_plot(hinge = False, ywx = None, window = False):
    plt.figure(figsize=(6, 3))
    ax = plt.axes()
    ax.set_xlim((-2, 2))
    ax.set_ylim((-0.5, 2))
    ax.axhline(y=0, color = 'k', linestyle = '--', linewidth = 0.5)
    ax.axvline(x=0, color = 'k', linestyle = '--', linewidth = 0.5)
    plt.title('Loss')
    plt.text(0.25, -0.2, r'$y * {\bf w} \cdot {\bf x}$')
    plt.yticks([0,1])
    plt.xticks([-1, 0, 1])
    if hinge:
        xvals = [-2, 0, 2]
        yvals = [2, 0, 0]
        if ywx is not None and window:
            xvals = [ywx - 0.2, ywx + 0.2]
            if xvals[0] > 0 and xvals[1] > 0:
                yvals = [0, 0]
            elif xvals[0] < 0 and xvals[1] < 0:
                yvals = [abs(xvals[0]), abs(xvals[1])]
            else:
                xvals = [xvals[0], 0, xvals[1]]
                yvals = [abs(xvals[0]), 0, 0]
        plt.plot(xvals, yvals, color = 'blue')
        if ywx is not None:
            yvals = [0, 0] if ywx > 0 else [0, abs(ywx)]
            plt.plot([ywx, ywx], yvals, '-o', color = 'black')

    else:
        xvals = [-2, 0, 0, 2]
        yvals = [1, 1, 0, 0]
        if ywx is not None and window:
            xvals = [ywx - 0.2, ywx + 0.2]
            if xvals[0] > 0 and xvals[1] > 0:
                yvals = [0, 0]
            elif xvals[0] < 0 and xvals[1] < 0:
                yvals = [1, 1]
            else:
                xvals = [xvals[0], 0, 0, xvals[1]]
        plt.plot(xvals, yvals, color = 'blue')
        if ywx is not None:
            yvals = [0, 0] if ywx > 0 else [0, 1]
            plt.plot([ywx, ywx], yvals, '-o', color = 'black')



In [33]:
def perceptron_loss_plot_3d(x, y):
    
    def lp(w1, w2):
        return max(-y * np.dot([w1, w2], x), 0)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    w1 = w2 = np.arange(-3.0, 3.0, 0.1)
    W1, W2 = np.meshgrid(w1, w2)
    zs = np.array([lp(w1, w2) for w1, w2 in zip(np.ravel(W1), np.ravel(W2))])
    Z = zs.reshape(W1.shape)

    ax.plot_surface(W1, W2, Z)

    ax.set_xlabel(r'$w_1$')
    ax.set_ylabel(r'$w_2$')
    ax.set_zlabel(r'$L_p$')

    plt.show()

In [34]:
def multiclass_perceptron(df, label = 'y', epochs = 100, bias = True):
    
    if bias:
        df = df.copy()
        df.insert(0, '_x0_', 1)
      
    w = {c:np.zeros(len(df.columns) - 1) for c in df[label].unique()}   
    features = [column for column in df.columns if column != label]

    for _ in range(epochs):
        errors = 0
        for _, row in df.iterrows():
            x = row[features]
            y = row[label]
            
            yhat = sorted(w.keys(), 
                          key = lambda c: np.dot(w[c], x), 
                          reverse = True)[0]
            
            if y != yhat:
                w[yhat] -= x
                w[y] += x
                errors += 1
            yield w.copy()
        if errors == 0:
            break